In [2]:
import pandas as pd
import numpy as np
import pickle
import ast
import operator
from sklearn.preprocessing import MultiLabelBinarizer
import re

 # Ranking DF

In [3]:
rank_df = pd.read_csv('/Users/KC/Downloads/boardgamegeek-reviews/2019-05-02.csv')

In [192]:
rank_df.columns

Index(['ID', 'Name', 'Year', 'Rank', 'Average', 'Bayes average', 'Users rated',
       'URL', 'Thumbnail'],
      dtype='object')

In [41]:
rank_df.head()

,ID,Name,Year,Rank,Average,Bayes average,Users rated,URL,Thumbnail
0,174430,Gloomhaven,2017,1,8.90,8.611,24574,/boardgame/174430/gloomhaven,https://cf.geekdo-images.com/micro/img/8JYMPXdcBg_UHddwzq64H4NBduY=/fit-in/64x64/pic2437871.jpg
1,161936,Pandemic Legacy: Season 1,2015,2,8.64,8.489,30921,/boardgame/161936/pandemic-legacy-season-1,https://cf.geekdo-images.com/micro/img/KbAKyhbG4qab4r-A_pBjUGvgal0=/fit-in/64x64/pic2452831.png
2,182028,Through the Ages: A New Story of Civilization,2015,3,8.53,8.258,15551,/boardgame/182028/through-ages-new-story-civilization,https://cf.geekdo-images.com/micro/img/APvZ_BYOt4ElpIXVl7i6wUp2BvM=/fit-in/64x64/pic2663291.jpg
3,167791,Terraforming Mars,2016,4,8.40,8.242,38331,/boardgame/167791/terraforming-mars,https://cf.geekdo-images.com/micro/img/71v8jF5vZFBqE64QJ7S3UYtDAng=/fit-in/64x64/pic3536616.jpg
4,12333,Twilight Struggle,2005,5,8.33,8.175,34268,/boardgame/12333/twilight-struggle,https://cf.geekdo-images.com/micro/img/uStZr7_8Tbthswp3sya1c2o1cs0=/fit-in/64x64/pic361592.jpg


# Game Attributes

In [4]:
games_df = pd.read_csv('/Users/KC/Downloads/boardgamegeek-reviews/games_detailed_info.csv',index_col=0)

/Users/KC/miniconda3/envs/metis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [103]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17063 entries, 0 to 17062
Data columns (total 55 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Abstract Game Rank             953 non-null    float64
 1   Accessory Rank                 1 non-null      float64
 2   Amiga Rank                     1 non-null      float64
 3   Arcade Rank                    1 non-null      float64
 4   Atari ST Rank                  1 non-null      float64
 5   Board Game Rank                17063 non-null  object 
 6   Children's Game Rank           767 non-null    float64
 7   Commodore 64 Rank              1 non-null      float64
 8   Customizable Rank              280 non-null    float64
 9   Family Game Rank               1806 non-null   float64
 10  Party Game Rank                525 non-null    float64
 11  RPG Item Rank                  1 non-null      float64
 12  Strategy Game Rank             1900 non-null  

In [5]:
games_df = games_df.drop(columns=['boardgameintegration'])

In [98]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17063 entries, 0 to 17062
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Abstract Game Rank             953 non-null    float64
 1   Accessory Rank                 1 non-null      float64
 2   Amiga Rank                     1 non-null      float64
 3   Arcade Rank                    1 non-null      float64
 4   Atari ST Rank                  1 non-null      float64
 5   Board Game Rank                17063 non-null  object 
 6   Children's Game Rank           767 non-null    float64
 7   Commodore 64 Rank              1 non-null      float64
 8   Customizable Rank              280 non-null    float64
 9   Family Game Rank               1806 non-null   float64
 10  Party Game Rank                525 non-null    float64
 11  RPG Item Rank                  1 non-null      float64
 12  Strategy Game Rank             1900 non-null  

In [82]:
games_df['num_artists'] = games_df['boardgameartist'].str.split().str.len()

In [84]:
games_df['num_expansions'] = games_df['boardgameexpansion'].str.split().str.len()

In [105]:
final_games = games_df[['average', 'averageweight', 'bayesaverage', 'boardgamecategory',
       'boardgamedesigner', 'boardgamemechanic', 'boardgamepublisher',
       'description', 'id', 'maxplayers', 'maxplaytime', 'median', 'minage',
       'minplayers', 'minplaytime', 'numcomments', 'numweights', 'owned',
       'playingtime', 'primary', 'stddev', 'wanting', 'wishing',
       'yearpublished']]

In [174]:
final_games.columns
# columns to keep
# 'id','average', 'averageweight', 'bayesaverage','median','playingtime', 'primary', 'stddev','wanting', 'wishing'

Index(['average', 'averageweight', 'bayesaverage', 'boardgamecategory',
       'boardgamedesigner', 'boardgamemechanic', 'boardgamepublisher',
       'description', 'id', 'maxplayers', 'maxplaytime', 'median', 'minage',
       'minplayers', 'minplaytime', 'numcomments', 'numweights', 'owned',
       'playingtime', 'primary', 'stddev', 'wanting', 'wishing',
       'yearpublished', 'num_artists', 'num_expansions'],
      dtype='object')

# Game Attributes - File 2

In [6]:
moregames = pd.read_csv('/Users/KC/Downloads/boardgames1.csv')

In [5]:
moregames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   objectid                20000 non-null  int64  
 1   name                    20000 non-null  object 
 2   yearpublished           20000 non-null  int64  
 3   sortindex               20000 non-null  int64  
 4   minplayers              20000 non-null  int64  
 5   maxplayers              20000 non-null  int64  
 6   minplaytime             20000 non-null  int64  
 7   maxplaytime             20000 non-null  int64  
 8   minage                  20000 non-null  int64  
 9   min_community           14141 non-null  float64
 10  max_community           14803 non-null  float64
 11  totalvotes              20000 non-null  int64  
 12  playerage               20000 non-null  object 
 13  languagedependence      20000 non-null  int64  
 14  usersrated              20000 non-null

# Creating Dummy Variables for Board Game Category

In [7]:
# list of unique board game categories
all_categories = moregames['boardgamecategory'].str.split(", ", expand=True).stack().str.strip("[]'").unique().tolist()

In [8]:
cat_tally = {}
for row in moregames['boardgamecategory']:
    categories = row.split(',')
    for each in categories:
        each = each.replace('[','').replace(']','').strip()
        if each in cat_tally:
            cat_tally[each] += 1
        else:
            cat_tally[each] = 1
    

In [9]:
sorted_categories = dict( sorted(cat_tally.items(), key=operator.itemgetter(1),reverse=True))

In [10]:
# looking at most popular categories
sorted_categories

{"'Card Game'": 5686,
 "'Wargame'": 3500,
 "'Fantasy'": 2361,
 "'Party Game'": 1723,
 "'Dice'": 1712,
 "'Fighting'": 1565,
 "'Abstract Strategy'": 1541,
 "'Childrens Game'": 1521,
 "'Science Fiction'": 1495,
 "'Economic'": 1423,
 "'Bluffing'": 1136,
 "'World War II'": 1129,
 "'Animals'": 1107,
 "'Humor'": 1064,
 "'Deduction'": 1046,
 "'Adventure'": 1017,
 "'Action / Dexterity'": 999,
 "'Movies / TV / Radio theme'": 980,
 "'Miniatures'": 925,
 "'Medieval'": 888,
 "'Exploration'": 792,
 "'Racing'": 709,
 "'Ancient'": 697,
 "'Negotiation'": 636,
 "'Real-time'": 623,
 "'Horror'": 593,
 "'Nautical'": 575,
 "'Trivia'": 572,
 "'Sports'": 520,
 "'Puzzle'": 511,
 "'Memory'": 495,
 "'Political'": 487,
 "'City Building'": 466,
 "'Word Game'": 462,
 "'Print & Play'": 462,
 "'Educational'": 442,
 "'Novel-based'": 437,
 "'Territory Building'": 416,
 "'Transportation'": 361,
 "'Collectible Components'": 342,
 "'Modern Warfare'": 331,
 "'Murder/Mystery'": 329,
 "'Civilization'": 319,
 "'Trains'": 318,

In [11]:
all_game_cats = []    
for row in moregames['boardgamecategory']:
    categories = row.split(',')
    game_cats = []
    for each in categories:
        each = each.replace('[','').replace(']','').replace("'",'').strip()
        game_cats.append(each)
    all_game_cats.append(game_cats)

In [12]:
top_categories = ['Card Game',
 'Wargame',
 'Fantasy',
 'Party Game',
 'Dice',
 'Fighting',
 'Abstract Strategy',
 'Childrens Game',
 'Science Fiction',
 'Economic',
 'Bluffing',
 'World War II',
 'Animals',
 'Humor',
 'Deduction',
 'Adventure',
 'Action / Dexterity',
 'Movies / TV / Radio theme',
 'Trivia',
 'Puzzle',
'Political',
 'City Building',
 'Word Game',
 'Print & Play',
 'Educational',
 'Novel-based',
 'Territory Building',
 'Transportation',
 'Collectible Components',
 'Modern Warfare',
 'Murder/Mystery',
 'Civilization',
 'Trains',
 'Mythology',
 'Pirates',
 'Napoleonic',
 'Comic Book / Strip',
 'Video Game Theme',
 'Industry / Manufacturing',
 'Aviation / Flight',
 'Renaissance',
 'Expansion for Base-game',
 'Farming',
 'Space Exploration',
 'American Civil War',
 'American West',
 'World War I',
 'Electronic',
 'Travel',
 'Zombies',
 'Maze',
 'Spies/Secret Agents']

In [13]:
# used MultiLabelBinarizer to create dummy variables for all game categories
mlb = MultiLabelBinarizer()
game_categories = pd.DataFrame(mlb.fit_transform(all_game_cats),columns=mlb.classes_, index=moregames.objectid)

In [14]:
# filter for only selected top categories
game_categories_df = game_categories[top_categories]

# Creating Dummy Variables for Designers

In [15]:
def dummy_vars(column):
    ''' function to turn a column from list of lists to one-hot encoded for each item in a list
    input: column
    return: dictionary with values sorted by most frequent, cleans each row to return only 1 list per row'''
    var_tally = {}
    all_games = []    
    for row in moregames[column]:
        # split into one list
        categories = row.split(',')
        game_categories = []
        for each in categories:
            # clean each row
            each = re.sub('[^A-Za-z ]+', '', each).strip()
            game_categories.append(each)
            if each in var_tally:
                # creates count for each value in the whole column
                var_tally[each] += 1
            else:
                var_tally[each] = 1
        all_games.append(game_categories)
    sorted_categories = dict( sorted(var_tally.items(), key=operator.itemgetter(1),reverse=True))
    return sorted_categories, all_games

In [16]:
sorted_designers, all_designers = dummy_vars('boardgamedesigner')

In [17]:
# looking at most popular designers
sorted_designers

{'Uncredited': 1776,
 'Reiner Knizia': 307,
 'Wolfgang Kramer': 135,
 'Joseph Miranda': 124,
 'Richard H Berg': 110,
 'James Ernest': 97,
 'Jim Dunnigan': 94,
 'Martin Wallace': 89,
 'Michael Schacht': 86,
 'Bruno Cathala': 84,
 'Frank Chadwick': 79,
 'Dean Essig': 79,
 'Klaus Teuber': 77,
 'Alan R Moon': 74,
 'Steve Jackson I': 73,
 'Ty Bomba': 67,
 'Uwe Rosenberg': 66,
 'Bruno Faidutti': 66,
 'Reinhard Staupe': 64,
 'Andrew Looney': 63,
 'Eric M Lang': 62,
 'Friedemann Friese': 62,
 'Alex Randolph': 61,
 'Mike Elliott': 58,
 'Leo Colovini': 55,
 'Garrett J Donner': 55,
 'Michael S Steer': 55,
 'Wargames': 54,
 'Gufcnter Burkhardt': 53,
 'Michael Kiesling': 50,
 'Inka Brand': 50,
 'Markus Brand': 50,
 'Sid Sackson': 49,
 'Mark Herman': 49,
 'Dan Verssen': 49,
 'Rob Daviau': 48,
 'Fruedueric Bey': 48,
 'Rufcdiger Dorn': 47,
 'Matt Hyra': 47,
 'Michael Bennighof': 45,
 'Stefan Dorra': 44,
 'Roberto Fraga': 42,
 'Aaron Weissblum': 40,
 'Richard Borg': 39,
 'Ludovic Maublanc': 39,
 'Cornu

In [18]:
top_designers = ['Reiner Knizia','Wolfgang Kramer','Joseph Miranda','Richard H Berg','James Ernest',
                 'Jim Dunnigan','Martin Wallace','Michael Schacht','Bruno Cathala','Frank Chadwick','Dean Essig','Klaus Teuber']

In [19]:
game_designers = pd.DataFrame(mlb.fit_transform(all_designers),columns=mlb.classes_, index=moregames.objectid)

In [20]:
game_design_df = game_designers.loc[:, game_designers.columns.isin(top_designers)]

# Creating Dummy Variables for Mechanics

In [21]:
sorted_mechanics, all_mechanics = dummy_vars('boardgamemechanic')

In [22]:
# looking at most popular mechanics
sorted_mechanics

{'Dice Rolling': 4461,
 'Hand Management': 3810,
 'Set Collection': 2532,
 'Hexagon Grid': 2132,
 'Variable Player Powers': 2071,
 'Card Drafting': 1571,
 'Tile Placement': 1559,
 'Roll  Spin and Move': 1526,
 'Modular Board': 1499,
 'Area Majority  Influence': 1364,
 'Cooperative Game': 1201,
 'Simulation': 1088,
 'Simultaneous Action Selection': 1053,
 'AuctionBidding': 1044,
 'Action Points': 1011,
 'Area Movement': 1001,
 'Memory': 848,
 'Grid Movement': 811,
 'TeamBased Game': 811,
 'Take That': 783,
 'Point to Point Movement': 768,
 'Push Your Luck': 720,
 'Pattern Building': 701,
 'Deck': 631,
 'Worker Placement': 626,
 'Card Game': 612,
 'Trading': 587,
 'Player Elimination': 559,
 'Secret Unit Deployment': 552,
 'Pickup and Deliver': 544,
 'Role Playing': 513,
 'Network and Route Building': 496,
 'Campaign  Battle Card Driven': 494,
 'Pattern Recognition': 473,
 'Betting and Bluffing': 406,
 'Storytelling': 377,
 'Voting': 376,
 'Action Queue': 363,
 'Tricktaking': 361,
 'Acti

In [23]:
top_mechanics = ['Hand Management','Set Collection','Hexagon Grid','Variable Player Powers'
                 ,'Card Drafting','Tile Placement','Roll  Spin and Move','Modular Board','Area Majority  Influence',
                 'Cooperative Game','Simulation','Simultaneous Action Selection','AuctionBidding','Action Points',
                 'Area Movement','Memory','Grid Movement','TeamBased Game','Take That','Point to Point Movement',
                 'Push Your Luck','Pattern Building','Deck','Worker Placement','Trading','Player Elimination',
                 'Secret Unit Deployment','Pickup and Deliver','Role Playing','Network and Route Building','Campaign  Battle Card Driven',
                 'Pattern Recognition','Betting and Bluffing','Storytelling']

In [24]:
mlb = MultiLabelBinarizer()
game_mechanics = pd.DataFrame(mlb.fit_transform(all_mechanics),columns=mlb.classes_, index=moregames.objectid)

In [25]:
game_mech_df = game_mechanics.loc[:, game_mechanics.columns.isin(top_mechanics)]

## Merging Dataframes into One Game DF

In [121]:
moregames.columns

Index(['objectid', 'name', 'yearpublished', 'sortindex', 'minplayers',
       'maxplayers', 'minplaytime', 'maxplaytime', 'minage', 'min_community',
       'max_community', 'totalvotes', 'playerage', 'languagedependence',
       'usersrated', 'average', 'baverage', 'stddev', 'avgweight',
       'numweights', 'numgeeklists', 'numtrading', 'numwanting', 'numcomments',
       'siteviews', 'numplays', 'numplays_month', 'news', 'blogs', 'weblink',
       'podcast', 'label', 'boardgamedesigner_cnt', 'boardgameartist_cnt',
       'boardgamepublisher_cnt', 'boardgamehonor_cnt', 'boardgamecategory_cnt',
       'boardgamemechanic_cnt', 'boardgameexpansion_cnt',
       'boardgameversion_cnt', 'boardgamefamily_cnt', 'boardgamedesigner',
       'boardgameartist', 'boardgamepublisher', 'boardgamehonor',
       'boardgamecategory', 'boardgameversion', 'boardgamemechanic',
       'boardgameexpansion', 'boardgamefamily', 'description', 'gamelink'],
      dtype='object')

In [26]:
# merge with designers dummy
moregames = moregames.merge(game_design_df, how='left', left_on='objectid', left_index=False, right_index=True)

In [27]:
# merge with categories dummy
moregames = moregames.merge(game_categories_df, how='left', left_on='objectid', left_index=False, right_index=True)

In [28]:
# merge with mechanics dummy
moregames = moregames.merge(game_mech_df, how='left', left_on='objectid', left_index=False, right_index=True)

In [135]:
moregames.columns[0:50]

Index(['objectid', 'name', 'yearpublished', 'sortindex', 'minplayers',
       'maxplayers', 'minplaytime', 'maxplaytime', 'minage', 'min_community',
       'max_community', 'totalvotes', 'playerage', 'languagedependence',
       'usersrated', 'average', 'baverage', 'stddev', 'avgweight',
       'numweights', 'numgeeklists', 'numtrading', 'numwanting', 'numcomments',
       'siteviews', 'numplays', 'numplays_month', 'news', 'blogs', 'weblink',
       'podcast', 'label', 'boardgamedesigner_cnt', 'boardgameartist_cnt',
       'boardgamepublisher_cnt', 'boardgamehonor_cnt', 'boardgamecategory_cnt',
       'boardgamemechanic_cnt', 'boardgameexpansion_cnt',
       'boardgameversion_cnt', 'boardgamefamily_cnt', 'boardgamedesigner',
       'boardgameartist', 'boardgamepublisher', 'boardgamehonor',
       'boardgamecategory', 'boardgameversion', 'boardgamemechanic',
       'boardgameexpansion', 'boardgamefamily'],
      dtype='object')

In [29]:
# moregames_df = moregames[['objectid', 'name', 'yearpublished','minplayers',
#        'maxplayers', 'minplaytime', 'maxplaytime', 'minage', 'min_community',
#        'max_community', 'playerage', 'languagedependence', 'avgweight',
#        'numweights', 'numgeeklists','news', 'blogs', 'weblink',
#        'podcast', 'boardgamedesigner_cnt', 'boardgameartist_cnt',
#         'boardgamecategory_cnt',
#        'boardgamemechanic_cnt','Reiner Knizia','Wolfgang Kramer','Joseph Miranda','Richard H Berg','James Ernest',
#     'Jim Dunnigan','Martin Wallace','Michael Schacht','Bruno Cathala','Frank Chadwick','Dean Essig','Hand Management','Set Collection','Hexagon Grid','Variable Player Powers'
#                  ,'Card Drafting','Tile Placement','Roll  Spin and Move','Modular Board','Area Majority  Influence',
#                  'Cooperative Game','Simulation','Simultaneous Action Selection','AuctionBidding','Action Points',
#                  'Area Movement','Memory','Grid Movement','TeamBased Game','Take That','Point to Point Movement',
#                  'Push Your Luck','Pattern Building','Deck','Worker Placement','Trading','Player Elimination',
#                  'Secret Unit Deployment','Pickup and Deliver','Role Playing','Network and Route Building','Campaign  Battle Card Driven',
#                  'Pattern Recognition','Betting and Bluffing','Storytelling'
# ]]

excl_col = ['sortindex', 'min_community',
       'max_community', 'totalvotes', 'usersrated','average', 'baverage','avgweight', 'stddev','numweights','numtrading', 'numwanting', 'numcomments',
        'numplays', 'numplays_month','label','boardgameexpansion_cnt',
       'boardgameversion_cnt', 'boardgamefamily_cnt','description', 'gamelink', 'boardgamehonor_cnt',
        'boardgameexpansion_cnt', 'boardgamehonor',
        'boardgameversion',
       'boardgameexpansion', 'boardgamefamily']
moregames_df = moregames[moregames.columns[~moregames.columns.isin(excl_col)]]
# 'news', 'blogs', 'weblink'
# 'boardgamedesigner','boardgameartist', 'boardgamepublisher','boardgamemechanic','boardgamecategory',

## Merging All Game Attributes - 1 DataFrame

In [30]:
# merge 2 game info datasets into 1 dataframe
info_df = pd.merge(moregames_df,games_df[['id','average', 'averageweight', 'bayesaverage','median','playingtime', 'primary', 'stddev','wanting', 'wishing','owned']],left_on='objectid', right_on='id', how='right')

In [32]:
# merge with 2019 ranking data
info_df = info_df.merge(rank_df[['ID','Rank']],left_on='id', right_on='ID', how='right')

In [31]:
# drop where objectid null
info_df = info_df[info_df['objectid'].notnull()]

In [33]:
# drop redundant columns
info_df = info_df.drop(columns=['objectid','name','ID'])

In [174]:
info_df.columns

Index(['yearpublished', 'minplayers', 'maxplayers', 'minplaytime',
       'maxplaytime', 'minage', 'playerage', 'languagedependence',
       'numgeeklists', 'siteviews',
       ...
       'id', 'average', 'averageweight', 'bayesaverage', 'median',
       'playingtime', 'primary', 'stddev', 'wanting', 'wishing'],
      dtype='object', length=125)

In [34]:
# # game info ready to pickle
# pickle.dump(info_df, open("gameinfo.pkl", "wb"))

# Text Reviews
* grouping text reviews by game id
* filtering for only games with at least 10 character text reviews

In [2]:
reviews = pd.read_csv('/Users/KC/Downloads/boardgamegeek-reviews/bgg-13m-reviews.csv',index_col=0)

/Users/KC/miniconda3/envs/metis/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13170073 entries, 0 to 3356499
Data columns (total 5 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user     object 
 1   rating   float64
 2   comment  object 
 3   ID       int64  
 4   name     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 602.9+ MB


In [4]:
reviews['user'].nunique()

289913

In [17]:
reviews = reviews[reviews['comment'].notnull()]

In [266]:
# creates joined list of all reviews per game
game_dict = {}
def pull_all_reviews(game_id, df):
    game = df[df['ID'] == game_id]
    game_review_list = game['comment'].tolist()
    game_reviews = ' '.join(game_review_list)
    if len(game_reviews) < 10:
        try:
            game_desc = game['description'][0]
            game_reviews = game_desc
        except:
            pass
    return game_reviews

In [6]:
game_ids = reviews['ID'].unique()

In [20]:
# created dictionary with game and all reviews as value
for i in game_ids:
    game_reviews = pull_all_reviews(i, reviews)
    game_dict[i] = game_reviews

In [21]:
pd.set_option('max_colwidth',150)
# changes index to game ID
review_df = pd.DataFrame.from_dict(game_dict, orient='index')
review_df.columns = ['comment']
review_df = review_df.sort_index()
review_df

,comment
1,"Play this game. Play this game a lot. hard to find enough players the older Moskito version is for 4 players only, the newer HiG version is for 3..."
2,"Among my favorite games to play with 4 players. Players take turns as the dealer and play five hands of different games in rotation, taking as man..."
3,Variable winning conditions make for great interaction and play. Have to stay on your feet at all time to keep this into account. Great tension. ...
4,"Great game!!! Tried with 2 players first. This game will really shine with the max 4 players. Bidding mechanism, building mechanism, random bui..."
5,Classic game of Finance and Risk. one of the best...try the 'Special Powers' too the best game. play online also. contact me at ICQ 100875646 Be...
...,...
271693,"Very smart and surprisingly deep tile laying game, which forces you to plan a few rounds ahead. Small family tile laying game."
271785,"Pure fun Surprisingly good dexterity game with excellent production. It's more challenging than it may seem at a first glance, which makes it good..."
271869,"If you prefer dice to cards, the news is good. Sushi Roll successfully translates the speed, simplicity and fun of its wildly popular predecessor ..."
271896,"Finally! The Firefly board game we all wanted it to be, and with Star Wars flavor. Great replay value and even a single player mode with the AI c..."


In [35]:
review_df = review_df[review_df['comment'].str.split().str.len() > 8]

In [96]:
# review_df ready to pickle & process
# pickle.dump(review_df, open("reviews.pkl", "wb"))

In [203]:
review_df['length'] = review_df['comment'].str.len()

/Users/KC/miniconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [207]:
review_df

,comment,length
1,"Play this game. Play this game a lot. hard to find enough players the older Moskito version is for 4 players only, the newer HiG version is for 3...",408812
2,"Among my favorite games to play with 4 players. Players take turns as the dealer and play five hands of different games in rotation, taking as man...",36815
3,Variable winning conditions make for great interaction and play. Have to stay on your feet at all time to keep this into account. Great tension. ...,583490
4,"Great game!!! Tried with 2 players first. This game will really shine with the max 4 players. Bidding mechanism, building mechanism, random bui...",19359
5,Classic game of Finance and Risk. one of the best...try the 'Special Powers' too the best game. play online also. contact me at ICQ 100875646 Be...,815998
...,...,...
271693,"Very smart and surprisingly deep tile laying game, which forces you to plan a few rounds ahead. Small family tile laying game.",126
271785,"Pure fun Surprisingly good dexterity game with excellent production. It's more challenging than it may seem at a first glance, which makes it good...",461
271869,"If you prefer dice to cards, the news is good. Sushi Roll successfully translates the speed, simplicity and fun of its wildly popular predecessor ...",2223
271896,"Finally! The Firefly board game we all wanted it to be, and with Star Wars flavor. Great replay value and even a single player mode with the AI c...",2425


In [219]:
# creating dataframe with game descriptions
text_df = pd.merge(reviews, moregames[['objectid','description']], how='left', left_on = 'ID', right_on= 'objectid')

In [267]:
# created dictionary with more text - all reviews & game description if comments are too low.
new_game_dict = {}
for i in game_ids:
    game_reviews = pull_all_reviews(i, text_df)
    new_game_dict[i] = game_reviews

In [268]:
pd.set_option('max_colwidth',150)
# changes index to trail names
alltext_df = pd.DataFrame.from_dict(new_game_dict, orient='index')
alltext_df.columns = ['comment']
alltext_df = alltext_df.sort_index()
alltext_df

,comment
1,"Play this game. Play this game a lot. hard to find enough players the older Moskito version is for 4 players only, the newer HiG version is for 3..."
2,"Among my favorite games to play with 4 players. Players take turns as the dealer and play five hands of different games in rotation, taking as man..."
3,Variable winning conditions make for great interaction and play. Have to stay on your feet at all time to keep this into account. Great tension. ...
4,"Great game!!! Tried with 2 players first. This game will really shine with the max 4 players. Bidding mechanism, building mechanism, random bui..."
5,Classic game of Finance and Risk. one of the best...try the 'Special Powers' too the best game. play online also. contact me at ICQ 100875646 Be...
...,...
271693,"Very smart and surprisingly deep tile laying game, which forces you to plan a few rounds ahead. Small family tile laying game."
271785,"Pure fun Surprisingly good dexterity game with excellent production. It's more challenging than it may seem at a first glance, which makes it good..."
271869,"If you prefer dice to cards, the news is good. Sushi Roll successfully translates the speed, simplicity and fun of its wildly popular predecessor ..."
271896,"Finally! The Firefly board game we all wanted it to be, and with Star Wars flavor. Great replay value and even a single player mode with the AI c..."


In [279]:
alltext_df = alltext_df[alltext_df['comment'].str.len() > 30]

In [280]:
# # alltext_df ready to pickle & process
# pickle.dump(alltext_df, open("alltext.pkl", "wb"))